In [58]:
import speech_recognition
import tempfile
from gtts import gTTS
import time
import wave
import pyaudio
from pygame import mixer
mixer.init()

def listenTo():
    
    CHUNK = 2048*20
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "test.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")
    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    r = speech_recognition.Recognizer()
    with speech_recognition.AudioFile('test.wav') as source:
        audio = r.record(source)
    t = time.time()
    s = r.recognize_google(audio, language='cmn-Hant-TW')
    print('recognize audio', time.time()-t)
    return s
    
def speak(sentence):
    t = time.time()
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts =gTTS(text=sentence, lang='zh-tw')
        tts.save(fp.name+'.mp3')
        mixer.music.load(fp.name+'.mp3')
        mixer.music.play()
    print('text to speech', time.time()-t)
# print(listenTo())

speak(listenTo())

* recording
* done recording
('read audio', 0.010174036026000977)
('recognize audio', 1.3876159191131592)
('text to speech', 1.1973979473114014)


In [45]:
r = speech_recognition.Recognizer()
r.recognize_google

In [24]:
for index, name in enumerate(speech_recognition.Microphone.list_microphone_names()):
    print(index, name)

(0, u'USB PnP Sound Device: Audio (hw:0,0)')
(1, u'bcm2835 ALSA: - (hw:1,0)')
(2, u'sysdefault')
(3, u'spdif')
(4, u'default')
